<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Init-data" data-toc-modified-id="Init-data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Init data</a></span><ul class="toc-item"><li><span><a href="#Estaciones" data-toc-modified-id="Estaciones-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Estaciones</a></span></li><li><span><a href="#Data-Mediciones" data-toc-modified-id="Data-Mediciones-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Data Mediciones</a></span></li></ul></li><li><span><a href="#Niveles-promedio-de-ozono-por-estación" data-toc-modified-id="Niveles-promedio-de-ozono-por-estación-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Niveles promedio de ozono por estación</a></span><ul class="toc-item"><li><span><a href="#Mplleaflet" data-toc-modified-id="Mplleaflet-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Mplleaflet</a></span></li><li><span><a href="#Folium" data-toc-modified-id="Folium-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Folium</a></span></li><li><span><a href="#Bokeh" data-toc-modified-id="Bokeh-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Bokeh</a></span></li></ul></li></ul></div>

In [35]:
import pandas as pd
import geopandas as gpd
from joblib import Parallel, delayed
import numpy as np
import json
from pathlib import Path
import altair as alt
from pandas.plotting import autocorrelation_plot
import matplotlib.pyplot as plt
import matplotlib as mpl
import plotly.plotly as py
import plotly.graph_objs as go
import mplleaflet

pd.options.mode.chained_assignment = None

%matplotlib inline

# Init data

## Estaciones

In [2]:
estaciones = pd.read_csv('/data/calidad_aire_presidencia/data_estaciones_sinaica.csv')
mask = (estaciones.lat.between(14, 34.5)) & (estaciones.long.between(-120, -70))
estaciones = estaciones[mask].rename(columns={'id':'estacionesid'})

print(estaciones.shape)
estaciones.head()

(180, 8)


,_id,lat,long,estacionesid,nombre,codigo,redesid,date-insert
0,5b8850e3e2705c1932e82ff8,21.873311,-102.320803,31,CBTIS,CBT,30,2018-08-30 20:17:39.218
1,5b8850e3e2705c1932e82ff9,21.846392,-102.288431,32,Secretaría de Medio Ambiente,SMA,30,2018-08-30 20:17:39.218
2,5b8850e3e2705c1932e82ffa,21.883781,-102.295825,33,Centro,CEN,30,2018-08-30 20:17:39.218
3,5b8850e3e2705c1932e82ffb,31.859917,-116.593722,36,Secundaria,SPABC20,31,2018-08-30 20:17:39.218
4,5b8850e3e2705c1932e82ffc,32.631317,-115.444631,38,UABC,SPABC12,32,2018-08-30 20:17:39.218


## Data Mediciones

In [3]:
mediciones = pd.read_csv('/data/calidad_aire_presidencia/data_mediciones_todas_estaciones_2.csv')
print(mediciones.shape)
mediciones.head()

(3240000, 8)


,city,estacionesid,fecha,hora,parametro,state,validoorig,valororig
0,Durango,58,2018-08-01,13,PM10,Durango,1,988.61
1,Durango,58,2018-08-01,14,PM10,Durango,1,988.61
2,Durango,58,2018-08-01,15,PM10,Durango,1,988.60
3,Durango,58,2018-08-01,16,PM10,Durango,1,988.63
4,Durango,58,2018-08-01,17,PM10,Durango,1,988.63


In [4]:
estaciones_ciudad = mediciones[['city', 'state', 'estacionesid']].drop_duplicates()
print(estaciones_ciudad.shape)
estaciones_ciudad.head()

(123, 3)


,city,state,estacionesid
0,Durango,Durango,58
6543,Durango,Durango,59
33554,Durango,Durango,60
65683,Gómez Palacio,Durango,65
80484,Celaya,Guanajuato,68


In [5]:
estaciones_ciudad.groupby('city').estacionesid.nunique().sort_values(ascending=False).head()

city
Valle de México    32
Monterrey          11
Guadalajara        10
Toluca              6
León                3
Name: estacionesid, dtype: int64

# Niveles promedio de ozono por estación

In [6]:
medicion_contaminante = mediciones[mediciones.parametro=='O3']

# De acuerdo al análisis anterior, es buena idea filtrar valores demasiado grandes. Vamos a usar
# como filtro el mínimo valor de ozono para establecer calidad del aire muy mala según SEMARNAT que es 0.6 ppm
medicion_contaminante = medicion_contaminante[medicion_contaminante.valororig.between(0, 0.7)]

medicion_contaminante = medicion_contaminante.groupby('estacionesid').agg({'valororig':['mean', 'count'],
                                                                           'validoorig':'mean'}).reset_index()
medicion_contaminante.columns = ['estacionesid', 'valororig', 'num_registros', 'validoorig']

medicion_contaminante = medicion_contaminante.merge(estaciones[['estacionesid', 'lat', 'long', 'nombre']],
                                                    on='estacionesid')
medicion_contaminante = medicion_contaminante.merge(mediciones[['estacionesid', 'city']].drop_duplicates(),
                                                    on='estacionesid')

print(medicion_contaminante.shape)
medicion_contaminante.head()

(107, 8)


,estacionesid,valororig,num_registros,validoorig,lat,long,nombre,city
0,33,0.290461,143,1,21.883781,-102.295825,Centro,Aguascalientes
1,38,0.039118,2,1,32.631317,-115.444631,UABC,Mexicali
2,39,0.017146,5858,1,32.639722,-115.506389,COBACH,Mexicali
3,41,0.010098,365,1,32.603639,-115.485944,CESPM,Mexicali
4,46,0.021328,9040,1,32.529461,-116.921281,Laboratorio,Tijuana


In [7]:
medicion_contaminante.describe()

,estacionesid,valororig,num_registros,validoorig,lat,long
count,107.000000,107.000000,107.000000,107.0,107.000000,107.000000
mean,179.158879,0.031631,4905.056075,1.0,21.379544,-101.007031
std,105.218564,0.026933,2006.699035,0.0,3.218338,3.670120
min,33.000000,0.000000,2.000000,1.0,17.094667,-116.921281
25%,84.500000,0.024084,4174.000000,1.0,19.442569,-101.658056
50%,137.000000,0.030195,4887.000000,1.0,20.120556,-99.683489
75%,259.500000,0.034239,5740.500000,1.0,21.125694,-99.155694
max,427.000000,0.290461,10456.000000,1.0,32.639722,-89.624811


In [ ]:
mask = (medicion_contaminante.num_registros>2400)
filtered_medicion = medicion_contaminante[mask]

print(filtered_medicion.shape)
filtered_medicion.head()

In [9]:
filtered_medicion.describe()

,estacionesid,valororig,num_registros,validoorig,lat,long
count,96.000000,96.000000,96.000000,96.0,96.000000,96.000000
mean,180.635417,0.029606,5346.187500,1.0,20.967580,-100.619271
std,99.778821,0.007263,1579.755113,0.0,2.840620,3.180649
min,39.000000,0.009592,2424.000000,1.0,17.094667,-116.921281
25%,89.250000,0.024368,4422.500000,1.0,19.401472,-101.380208
50%,137.500000,0.030665,4994.500000,1.0,19.956111,-99.568431
75%,259.250000,0.034222,5852.750000,1.0,20.694510,-99.117778
max,425.000000,0.046619,10456.000000,1.0,32.639722,-89.624811


## Folium

In [162]:
import folium
from shapely.geometry import Point
import branca.colormap as cm
import matplotlib.cm as mtplb_cm
import matplotlib.colors as mcolors

In [196]:
centro_lat, centro_lon = 22.396092, -101.731430
folium_map = folium.Map(location=[centro_lat, centro_lon], zoom_start=5)

colormap = cm.LinearColormap(list(map(mcolors.to_hex, mtplb_cm.viridis.colors)),
                             vmin=filtered_medicion.valororig.min(),
                             vmax=filtered_medicion.valororig.max()).to_step(10)

colormap.caption = 'Valor promedio de O3 por estacion'

filtered_medicion['color'] = filtered_medicion.valororig.map(colormap)

for i, row in filtered_medicion.iterrows():
    
    # generate the popup message that is shown on click.
    popup_text = "<b> Valor O3 promedio: </b> %.03f <br>" %row.valororig
    popup_text += f" <b> Conteo de registros para O3: </b> {row.num_registros} <br> \
    <b> Nombre: </b> {row.nombre} <br> <b> Ciudad: </b> {row.city} <br>"
    
    folium.CircleMarker(location=[row.lat, row.long], radius=4, tooltip=popup_text,
                        color=row.color, fill=True, fill_opacity=0.6).add_to(folium_map)
    
folium_map.add_child(colormap)    

# folium_map.save('../mapa_niveles_de_ozono_promedio_estaciones.html')

folium_map

In [190]:
from folium.plugins import HeatMap

aux = filtered_medicion[filtered_medicion.city=='Valle de México']

m = folium.Map(location=[aux.lat.median(), aux.long.median()], zoom_start=10,
                        tiles="CartoDB positron")

HeatMap(aux[['lat', 'long', 'valororig']].values).add_to(m)

m